In [ ]:
import simpy
import pandas as pd

In [ ]:
class Schedule(simpy.Environment):
    def __init__(self, name = "Test", Tasks=None,num_workers=simpy.core.Infinity, **meta):
        super().__init__()
        self.env=simpy.Environment()
        self.name=name
        self.meta = meta
        self._tasks = []
        self.make_task_df()
        self.workers = simpy.Resource(self, capacity=num_workers)
    def add_tasks(self, tasks):

        if not isinstance(tasks, list): tasks = [tasks]
        for task in tasks:
            
            self._tasks.append(task)
            self.process(task.do_task(self))
            task.complete = simpy.Event(self)
        
        self.env.run() 
            
    def make_task_df(self):
        tmp = pd.DataFrame()
        for task in self._tasks:
            tmp = tmp.append(
                pd.DataFrame({
                "ProjectName": [self.name],
                "ProjectMeta": [self.meta],    
                "TaskName": [task.name],
                "StartDate": [task.start_date],
                    "EndDate": [task.end_date],
                    "Preds": [[pred.name for pred in task.preds]],
                    "Meta": [task.meta]
                }
            ),  ignore_index=True) 
        self._tasks_df = tmp
        return self._tasks_df

    
class Task:
    def __init__(self,name, duration, preds, env, **meta ):
        self.meta = meta
        self.env=env
        self.name=name
        self.duration =duration
        if preds is None:
            self.preds = []
        else:
            if not isinstance(preds,list): 
                preds = [preds]
            self.preds = preds
        self.start_date = None
        self.end_date=None
        self.complete=simpy.Event(env)
        self.progress = 0

        
    def do_task(self,env):
        while not self.complete.triggered:
            # start task when preds begin
            for pred in self.preds:
                try:
                #print(self.name, "waiting on task", pred.name)
                    yield pred.complete
                except:
                    print(pred, "Something went wrong")
            #getting resources
            with env.workers.request() as request:
                yield request
                print(env.workers.count)
                print(self.name, 'starting task')
                self.start_date = env.now

                yield env.timeout(self.duration)
                self.end_date = env.now
                yield self.complete.succeed()
                print(self.name,'finished task')
                       
            #release resources
            #yield env.workers.release(1)
            
        

In [ ]:
S = Schedule(num_workers=5)
T1 = Task( 'Task1', 10, None, S, Important=True)
T2 = Task( 'Task2', 10, T1, S)
T3 = Task('Task3', 20, T1,S)
T4 = Task('Task4', 7, [T2,T3],S)
S.add_tasks([T1, T2,T3,T4])


In [ ]:
for task in S._tasks:
    print(task.name, task.start_date, task.end_date, task.meta)
S.make_task_df()

Task1 None None {'Important': True}
Task2 None None {}
Task3 None None {}
Task4 None None {}


,ProjectName,ProjectMeta,TaskName,StartDate,EndDate,Preds,Meta
0,Test,{},Task1,None,None,[],{'Important': True}
1,Test,{},Task2,None,None,[Task1],{}
2,Test,{},Task3,None,None,[Task1],{}
3,Test,{},Task4,None,None,"[Task2, Task3]",{}


In [ ]:
S.add_tasks(Task("New Task2", 50,[T1], S))
S.run()

1
New Task2 starting task
New Task2 finished task


In [ ]:
for task in S._tasks:
    print(task.name, task.start_date, task.end_date)
S.make_task_df()

Task1 0 10
Task2 10 20
Task3 10 30
Task4 30 37
New Task2 10 60
New Task2 60 110
New Task2 110 160
New Task2 160 210


,ProjectName,ProjectMeta,TaskName,StartDate,EndDate,Preds,Meta
0,Test,{},Task1,0,10,[],{'Important': True}
1,Test,{},Task2,10,20,[Task1],{}
2,Test,{},Task3,10,30,[Task1],{}
3,Test,{},Task4,30,37,"[Task2, Task3]",{}
4,Test,{},New Task2,10,60,[Task1],{}
5,Test,{},New Task2,60,110,[Task1],{}
6,Test,{},New Task2,110,160,[Task1],{}
7,Test,{},New Task2,160,210,[Task1],{}


In [ ]:
T3.preds

In [ ]:
T3.preds[0].name

In [ ]:
Employees = simpy.Resource(S,100000, init=100)

TypeError: __init__() got an unexpected keyword argument 'init'

In [ ]:
Employees.get(1)
Employees.request(
Employees.capacity

TypeError: __init__() takes 2 positional arguments but 3 were given

In [ ]:
S.workers.count

0